# Stable Diffusion Benchmark

## Install dependencies

In [1]:
%pip install aioboto3 boto3 pandas plotly requests pillow tabulate kaleido nbformat ipython python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Load the env and libraries

In [2]:
from utils import queue_jobs, salad_org_id, salad_project_name, reporting_api_key, reporting_url, queue_service_url, salad_headers, salad_api_base_url, delete_all_container_groups, start_all_container_groups, stop_all_container_groups, deep_merge, get_signed_upload_url, purge_all_queues, delete_all_container_groups_with_status, format_gpu_name, delete_empty_queues, list_all_container_groups
import requests
import os
import dotenv
import re
import copy
import uuid
import itertools
dotenv.load_dotenv(".env", override=True)


True

## Set up container groups

In [3]:
vcpu = 2
memory = 1024 * 12

replica_count_per_group = 5

create_container_group_payload = {
  "name": "replace-this",
  "replicas": replica_count_per_group,
  "autostart_policy": False,
  "container": {
    "image": "replaceme:latest",
    "resources": {
      "cpu": vcpu,
      "memory": memory,
      "gpu_classes": []
    },
    "environment_variables": {
        "REPORTING_API_KEY": reporting_api_key,
        "REPORTING_URL": reporting_url,
        "QUEUE_URL": queue_service_url,
        "QUEUE_API_KEY": reporting_api_key,
        "STARTUP_CHECK_MAX_TRIES": "1000"
    }
  }
}


def get_gpu_classes():
    url = f"{salad_api_base_url}/organizations/{salad_org_id}/gpu-classes"
    response = requests.get(url, headers=salad_headers)
    return [gpu for gpu in response.json()["items"] if gpu["name"] != "Stable Diffusion Compatible"]


def create_container_group(name, image, gpu, env={}, dry_run=False):
    payload = copy.deepcopy(create_container_group_payload)
    payload["name"] = name
    payload["container"]["image"] = image
    payload["container"]["resources"]["gpu_classes"] = [gpu]
    payload["container"]["environment_variables"].update(env)
    url = f"{salad_api_base_url}/organizations/{salad_org_id}/projects/{salad_project_name}/containers"

    if dry_run:
        print(url)
        print(payload)
        return
    response = requests.post(url, headers=salad_headers, json=payload)
    return response.json()


gpu_classes = get_gpu_classes()


images = {
  "stable-fast": {
    "baked": "saladtechnologies/stable-fast-qr-code:worker0.1.0-0.5.0-baked",
    "dynamic": "saladtechnologies/stable-fast-qr-code:worker0.1.0-0.5.0"
  },
  "sdnext": {
    "baked": "saladtechnologies/sdnext:worker0.1.0-122143-128713",
    "dynamic": "saladtechnologies/sdnext:worker0.1.0-dynamic"
  },
  "a1111": {
    "baked": "saladtechnologies/a1111:worker0.1.0-122143-128713",
    "dynamic": "saladtechnologies/a1111:worker0.1.0-dynamic"
  },
  "comfy": {
    "baked": "saladtechnologies/comfyui:worker0.1.0-baked",
    "dynamic": "saladtechnologies/comfyui:worker0.1.0-dynamic"
  },
}

env = {
  "stable-fast": {
    "baked": {},
    "dynamic": {
      "CIVITAI_CONTROLNET_MODEL": "122143",
      "CIVITAI_CHECKPOINT_MODEL": "128713"
    }
  },
  "sdnext": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
  "a1111": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
  "comfy": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
}

queue_names = []
container_groups = []

# delete_all_container_groups()

for i, gpu in enumerate(gpu_classes):
    print(i, gpu["name"])

attempt = 0

0 GTX 1650 (4 GB)
1 RTX 4080 (16 GB)
2 RTX 2070 (8 GB)
3 RTX 3060 Ti (8 GB)
4 RTX 2080 (8 GB)
5 GTX 1660 (6 GB)
6 RTX 4070 Ti (12 GB)
7 RTX 3060 (12 GB)
8 GTX 1050 Ti (4 GB)
9 RTX 2080 Ti (11 GB)
10 GTX 1660 Super (6 GB)
11 RTX 3080 Ti (12 GB)
12 GTX 1060 (6 GB)
13 RTX 3050 (8 GB)
14 RTX 4070 (12 GB)
15 RTX 3080 (10 GB)
16 RTX 2060 (6 GB)
17 GTX 1070 (8 GB)
18 RTX 4090 (24 GB)
19 RTX 3090 Ti (24 GB)
20 RTX 3090 (24 GB)
21 RTX 3070 Ti (8 GB)
22 RTX 3070 (8 GB)


## Create All of the Container Groups

In [4]:

def all_combos():
  for gpu in gpu_classes:
    for image_name, image in images.items():
      for baked in ["baked", "dynamic"]:
        for image_size in ["512", "768"]:
          yield gpu, image_name, image, baked, image_size

def one_gpu_class(gpu_index=0, resolutions=["512"], baked_variants=["baked", "dynamic"]):
  for image_name, image in images.items():
    for baked in baked_variants:
      for image_size in resolutions:
        yield gpu_classes[gpu_index], image_name, image, baked, image_size

def create_all_container_groups(combos):
  global attempt
  for gpu, image_name, image, baked, image_size in combos:
      env_vars = env[image_name][baked]
      name = f"{image_name}-{format_gpu_name(gpu['name'])}-{baked}-{image_size}"
      env_vars["QUEUE_NAME"] = name
      env_vars["BENCHMARK_ID"] = name
      env_vars["IMAGE_SIZE"] = image_size
      print(f"Creating container group {name}...")
      queue_names.append(name)
      create_container_group(f"{name}-{attempt}", image[baked], gpu["id"], env=env_vars)
      container_groups.append(f"{name}-{attempt}")

# stop_all_container_groups()
# delete_all_container_groups()
# delete_all_container_groups_with_status("stopped")
# attempt += 1


############ 1xxx cards ##############
# GTX 1050 Ti (6 GB)
# create_all_container_groups(one_gpu_class(8, resolutions=["512","768"]))

# GTX 1060 (6 GB)
# create_all_container_groups(one_gpu_class(12, resolutions=["512","768"]))

# GTX 1070 (8 GB)
# create_all_container_groups(one_gpu_class(17, resolutions=["512","768"]))

# GTX 1650 (4GB)
# create_all_container_groups(one_gpu_class(0, resolutions=["512"]))
# create_all_container_groups(one_gpu_class(0, resolutions=["768"]))

# GTX 1660 (6 GB)
# create_all_container_groups(one_gpu_class(5, resolutions=["512"]))
# create_all_container_groups(one_gpu_class(5, resolutions=["768"]))

# GTX 1660 Super (6 GB)
# create_all_container_groups(one_gpu_class(10, resolutions=["512","768"]))


############ 2xxx cards ##############
# RTX 2060 (6 GB)
# create_all_container_groups(one_gpu_class(16, resolutions=["512","768"]))

# GTX 2070 (8 GB)
# create_all_container_groups(one_gpu_class(2, resolutions=["512","768"]))

# RTX 2080 (8 GB)
# create_all_container_groups(one_gpu_class(4, resolutions=["512","768"])) # running

# RTX 2080 Ti (11 GB)
# create_all_container_groups(one_gpu_class(9, resolutions=["512","768"]))


############ 3xxx cards ##############
# RTX 3050 (8 GB)
# create_all_container_groups(one_gpu_class(13, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(13, resolutions=["768"])) # running

# RTX 3060 (12 GB)
# create_all_container_groups(one_gpu_class(7, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(7, resolutions=["768"])) #

# RTX 3060 Ti (8 GB)
# create_all_container_groups(one_gpu_class(3, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(3, resolutions=["768"])) #

# RTX 3070 (8 GB)
# create_all_container_groups(one_gpu_class(22, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(22, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(22, resolutions=["768"], baked_variants=["dynamic"])) 

# RTX 3070 Ti (8 GB)
# create_all_container_groups(one_gpu_class(21, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(21, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(21, resolutions=["768"], baked_variants=["dynamic"]))

# RTX 3080 (10 GB)
# create_all_container_groups(one_gpu_class(15, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(15, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(15, resolutions=["768"], baked_variants=["dynamic"]))

# RTX 3080 Ti (12 GB)
# create_all_container_groups(one_gpu_class(11, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(11, resolutions=["768"])) #

# RTX 3090 (24 GB)
# create_all_container_groups(one_gpu_class(20, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(20, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(20, resolutions=["768"], baked_variants=["dynamic"])) 

# RTX 3090 Ti (24 GB)
# create_all_container_groups(one_gpu_class(19, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(19, resolutions=["768"], baked_variants=["baked"])) # running
# create_all_container_groups(one_gpu_class(19, resolutions=["768"], baked_variants=["dynamic"])) 


############ 4xxx cards ##############
# RTX 4070 (12 GB)
# create_all_container_groups(one_gpu_class(14, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(14, resolutions=["768"])) #

# RTX 4070 Ti (12 GB)
# create_all_container_groups(one_gpu_class(6, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(6, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(6, resolutions=["768"], baked_variants=["dynamic"]))

# RTX 4080 (16 GB)
# create_all_container_groups(one_gpu_class(1, resolutions=["512"])) #
# create_all_container_groups(one_gpu_class(1, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(1, resolutions=["768"], baked_variants=["dynamic"]))

# RTX 4090 (24 GB)
# create_all_container_groups(one_gpu_class(18, resolutions=["512"], baked_variants=["baked"])) #
# create_all_container_groups(one_gpu_class(18, resolutions=["512"], baked_variants=["dynamic"])) #
# create_all_container_groups(one_gpu_class(18, resolutions=["768"], baked_variants=["baked"]))
# create_all_container_groups(one_gpu_class(18, resolutions=["768"], baked_variants=["dynamic"]))


start_all_container_groups()

print(f"{len(queue_names)} queues to be filled")

Creating container group stable-fast-rtx3090ti24gb-baked-768...
Creating container group sdnext-rtx3090ti24gb-baked-768...
Creating container group a1111-rtx3090ti24gb-baked-768...
Creating container group comfy-rtx3090ti24gb-baked-768...
Found 20 container groups.
4 queues to be filled


In [5]:
base_qr_payload = {
    "batch_size": 1,
    "upload_url": [],
    "stable_diffusion_params": {
        "controlnet_conditioning_scale": 2.0,
        "guidance_scale": 4.0,
        "control_guidance_start": 0.1,
        "control_guidance_end": 0.95,
        "negative_prompt": ""
    },
    "qr_params": {
        "drawer": "RoundedModule",
        "error_correction": "H",
        "color_mask": "SolidFill",
        "color_mask_params": {"front_color": [0, 0, 0], "back_color": [127, 127, 127]},
    },
}

variants = [
    {
        "stable_diffusion_params": {
            "prompt": "leafy green salad",
        },
        "qr_params": {
            "data": "https://salad.com",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "clouds",
        },
        "qr_params": {
            "data": "https://salad.com",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "ocean",
        },
        "qr_params": {
            "data": "https://salad.com",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "Pizza Pepperoni",
            "control_guidance_end": 1.0,
            "control_guidance_start": 0.05,
        },
        "qr_params": {
            "data": "https://salad.com/",
            "color_mask_params": {
              "back_color": [184, 184, 184]
            }
        },
    },
    {
        "stable_diffusion_params": {"prompt": "fruit salad", "guidance_scale": 3.8},
        "qr_params": {
            "data": "https://salad.com/pricing",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "leafy green salad",
            "guidance_scale": 3.7,
        },
        "qr_params": {
            "data": "https://salad.com/pricing",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "Pizza Pepperoni",
            "control_guidance_end": 1.0,
            "control_guidance_start": 0.05,
        },
        "qr_params": {
            "data": "https://salad.com/pricing",
            "color_mask_params": {
              "back_color": [184, 184, 184]
            }
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "fire",
        },
        "qr_params": {
            "data": "https://salad.com/download",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "galaxy",
        },
        "qr_params": {
            "data": "https://salad.com/download",
        },
    },
    {
        "stable_diffusion_params": {
            "prompt": "gold coins",
        },
        "qr_params": {
            "data": "https://salad.com/download",
        },
    },
]

# This will get run for every container group
def get_all_jobs(num_per_variant=10):
    for variant in variants:
        base = copy.deepcopy(base_qr_payload)
        payload = deep_merge(base, variant)
        for num_steps in [15, 50]:
            for batch_size in [1,2,4]:
                for i in range(num_per_variant):
                    job = copy.deepcopy(payload)
                    job["id"] = str(uuid.uuid4())
                    job["stable_diffusion_params"]["num_inference_steps"] = num_steps
                    job["batch_size"] = batch_size
                    yield job

jobs = list(get_all_jobs())
num_images = sum([job["batch_size"] for job in jobs])
print(f"Created {len(jobs)} jobs for {num_images} images.")



Created 600 jobs for 1400 images.


In [6]:
def get_all_jobs_for_backend(backend: str = "stable-fast"):
  image_ext = "jpg"
  image_type = "image/jpeg"
  if backend == "comfy":
    image_ext = "png"
    image_type = "image/png"
  for job in get_all_jobs():
    for i in range(job["batch_size"]):
      job["upload_url"].append(get_signed_upload_url(f"{job['id']}-{i}.{image_ext}", image_type))
    yield job


def get_backend_from_queue_name(queue_name: str):
  """
  stable-fast-gtx16504gb-baked-512 -> stable-fast
  a1111-rtx309024gb-dynamic-768 -> a1111
  """
  queue_name_parts = queue_name.split("-")
  return "-".join(queue_name_parts[:-3])

# await purge_all_queues(queue_names)

for queue_name in queue_names:
  # if "512" in queue_name:
  #   continue
  backend = get_backend_from_queue_name(queue_name)
  print(f"Queueing {backend} jobs for {queue_name}...")
  await queue_jobs(queue_name, get_all_jobs_for_backend(backend), delay=0.5)


Queueing stable-fast jobs for stable-fast-rtx3090ti24gb-baked-768...
{'status': 'No Messages Found', 'messages': []}
Queueing jobs for queue benchmark-stable-fast-rtx3090ti24gb-baked-768.fifo...
Sent 600 jobs in total.
Queueing sdnext jobs for sdnext-rtx3090ti24gb-baked-768...
{'status': 'No Messages Found', 'messages': []}
Queueing jobs for queue benchmark-sdnext-rtx3090ti24gb-baked-768.fifo...
Sent 600 jobs in total.
Queueing a1111 jobs for a1111-rtx3090ti24gb-baked-768...
{'status': 'No Messages Found', 'messages': []}
Queueing jobs for queue benchmark-a1111-rtx3090ti24gb-baked-768.fifo...
Sent 600 jobs in total.
Queueing comfy jobs for comfy-rtx3090ti24gb-baked-768...
{'status': 'No Messages Found', 'messages': []}
Queueing jobs for queue benchmark-comfy-rtx3090ti24gb-baked-768.fifo...
Sent 600 jobs in total.


In [8]:
# def get_benchmark_ids(combos):
#   for gpu, image_name, image, baked, image_size in combos:
#     yield f"{image_name}-{format_gpu_name(gpu['name'])}-{baked}-{image_size}"

# # Write all benchmark ids to a file
# with open("benchmark_ids.txt", "w") as f:
#   for benchmark_id in get_benchmark_ids(all_combos()):
#     f.write(f"{benchmark_id}\n")
await delete_empty_queues()

def sort_by_gpu_class(data):
    # Define a function to extract the GPU class
    def gpu_class_key(item):
        name = item["name"]
        match = re.search(r'\wtx\d+(ti)?\d+gb', name)
        return match.group(0) if match else ""

    # Sort the list using the GPU class as the key
    sorted_data = sorted(data, key=gpu_class_key)
    return sorted_data


total_running = 0
total_allocating = 0
total_creating = 0
for container_group in sort_by_gpu_class(list_all_container_groups("running")):
    name = container_group["name"]
    running_count = container_group["current_state"]["instance_status_count"]["running_count"]
    allocating_count = container_group["current_state"]["instance_status_count"]["allocating_count"]
    creating_count = container_group["current_state"]["instance_status_count"]["creating_count"]
    total_running += running_count
    total_allocating += allocating_count
    total_creating += creating_count
    print(
        f"{name} - {running_count} running - {creating_count} creating - {allocating_count} allocating"
    )
print(f"Total running: {total_running}")
print(f"Total allocating: {total_allocating}")
print(f"Total creating: {total_creating}")

Deleting queue benchmark-a1111-rtx20808gb-baked-768.fifo...
Deleting queue benchmark-comfy-rtx409024gb-dynamic-512.fifo...
Deleting queue benchmark-sdnext-rtx409024gb-dynamic-512.fifo...
Deleting queue benchmark-stable-fast-rtx409024gb-dynamic-512.fifo...
Found 12 container groups.
comfy-rtx20808gb-dynamic-768-1 - 1 running - 0 creating - 4 allocating
sdnext-rtx20808gb-baked-512-1 - 1 running - 2 creating - 2 allocating
sdnext-rtx20808gb-dynamic-512-1 - 0 running - 0 creating - 5 allocating
sdnext-rtx20808gb-dynamic-768-1 - 0 running - 0 creating - 5 allocating
stable-fast-rtx20808gb-baked-768-1 - 1 running - 1 creating - 3 allocating
a1111-rtx30508gb-dynamic-768-0 - 3 running - 0 creating - 2 allocating
comfy-rtx30508gb-baked-768-0 - 0 running - 0 creating - 5 allocating
comfy-rtx30508gb-dynamic-768-0 - 0 running - 0 creating - 5 allocating
sdnext-rtx30508gb-baked-768-0 - 1 running - 0 creating - 4 allocating
sdnext-rtx30508gb-dynamic-768-0 - 3 running - 1 creating - 1 allocating
stab